In [5]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pickle
from pathlib import Path

data_path = Path("../data/")   # full path to the HEF folder

train = pd.read_csv(data_path / "mimic_train_HEF.csv", low_memory=False)
test  = pd.read_csv(data_path / "mimic_test_HEF.csv",  low_memory=False)

train.shape, test.shape


((20885, 44), (5221, 39))

In [6]:
# =============================================================================
# 2. DROP LEAKAGE COLUMNS
# =============================================================================
columns_to_drop = [
    'DISCHTIME', 'DEATHTIME', 'DOD', 'LOS',
    'subject_id', 'hadm_id', 'icustay_id',
    'ADMITTIME', 'Diff'
]

train_clean = train.drop(columns=columns_to_drop, errors='ignore')
test_clean = test.drop(columns=columns_to_drop, errors='ignore')

In [7]:
# =============================================================================
# 3. SEPARATE TARGET
# =============================================================================
y = train_clean['HOSPITAL_EXPIRE_FLAG']
X = train_clean.drop('HOSPITAL_EXPIRE_FLAG', axis=1)
X_test = test_clean.copy()

In [8]:
# =============================================================================
# 4. IDENTIFY FEATURE TYPES
# =============================================================================
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nNumeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")


Numeric features: 24
Categorical features: 10


In [9]:
# =============================================================================
# 5. IMPUTATION
# =============================================================================
print("\n--- Imputing missing values ---")

# Numeric
numeric_imputer = SimpleImputer(strategy='median')
X[numeric_features] = numeric_imputer.fit_transform(X[numeric_features])
X_test[numeric_features] = numeric_imputer.transform(X_test[numeric_features])

# Categorical
categorical_imputer = SimpleImputer(strategy='most_frequent')
X[categorical_features] = categorical_imputer.fit_transform(X[categorical_features])
X_test[categorical_features] = categorical_imputer.transform(X_test[categorical_features])


--- Imputing missing values ---


In [10]:
# =============================================================================
# 6. ENCODING
# =============================================================================
print("--- Encoding categorical variables ---")

X_combined = pd.concat([X, X_test], keys=['train', 'test'])
X_encoded = pd.get_dummies(X_combined, columns=categorical_features, drop_first=True)
X = X_encoded.xs('train')
X_test = X_encoded.xs('test')

--- Encoding categorical variables ---


In [11]:
# =============================================================================
# 7. FEATURE SCALING
# =============================================================================
print("--- Scaling numeric features ---")

numeric_cols_to_scale = [col for col in numeric_features if col in X.columns]
scaler = StandardScaler()
X[numeric_cols_to_scale] = scaler.fit_transform(X[numeric_cols_to_scale])
X_test[numeric_cols_to_scale] = scaler.transform(X_test[numeric_cols_to_scale])

print(f"\nFinal shapes:")
print(f"  X: {X.shape}")
print(f"  y: {y.shape}")
print(f"  X_test: {X_test.shape}")

--- Scaling numeric features ---

Final shapes:
  X: (20885, 25782)
  y: (20885,)
  X_test: (5221, 25782)


C:\Users\corne\AppData\Local\Temp\ipykernel_21732\613314864.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_cols_to_scale] = scaler.fit_transform(X[numeric_cols_to_scale])
C:\Users\corne\AppData\Local\Temp\ipykernel_21732\613314864.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[numeric_cols_to_scale] = scaler.transform(X_test[numeric_cols_to_scale])


In [12]:
# =============================================================================
# 8. SAVE PROCESSED DATA
# =============================================================================
print("\n--- Saving processed data ---")

import os
os.makedirs('../data/processed', exist_ok=True)

# Save as pickle (preserves dtypes and column names)
X.to_pickle('../data/processed/X_train_processed.pkl')
y.to_pickle('../data/processed/y_train.pkl')
X_test.to_pickle('../data/processed/X_test_processed.pkl')
test_ids.to_pickle('../data/processed/test_ids.pkl')

# Also save preprocessing objects (to use on new data if needed)
with open('../data/processed/numeric_imputer.pkl', 'wb') as f:
    pickle.dump(numeric_imputer, f)
with open('../data/processed/categorical_imputer.pkl', 'wb') as f:
    pickle.dump(categorical_imputer, f)
with open('../data/processed/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("✓ Processed data saved to ../data/processed/")
print("\n" + "="*70)
print("PREPROCESSING COMPLETE!")
print("="*70)
print("\nYou can now run modeling notebooks without repeating preprocessing.")


--- Saving processed data ---


NameError: name 'test_ids' is not defined